<a href="https://colab.research.google.com/github/Ciaran-Jones/Swim-Meet-Analytics-System/blob/main/SYS2202_Final_Project(Ciaran_Jones).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Starting Section

## Library Import

In [ ]:
!pip install PyPDF2
!pip install tika
!pip install dash
from tika import parser
from PyPDF2 import PdfReader
from google.colab import files
import pandas as pd
import re
import io
import numpy as np
import datetime as dt
from datetime import datetime
from dash import Dash, html, dcc, Input, Output, callback
import plotly.graph_objects as go
import plotly.express as px

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


## Establishing The Connection

In [ ]:
!pip install pymysql

from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from sqlalchemy import text


# parameters
# !!! Please change below information with your local information
host_ip = "3.15.225.248" # Use your own EC2 instance IP address.
id = "test1" # Use your 'test1' username you created in Lab 10.
pw = "Test1234#"  # Use your 'Test1234#' password you created in Lab 10.

# connect to mysql server
url = URL.create(
    drivername="mysql+pymysql",
    host=host_ip,
    port=3306,
    username= id,
    password=pw)

sqlEngine = create_engine(url)
sql_connection = sqlEngine.connect()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.4 MB/s eta 0:00:00


# Creating the SQL Database

In [ ]:
# Make a new database. For your project, make sure you only run this step once at the beginning. You must be able to see the database in Workbench after running this command.
sql_connection.execute(text('DROP SCHEMA IF EXISTS meet_program_db'))
sql_connection.execute(text('CREATE SCHEMA IF NOT EXISTS meet_program_db;'))
sql_connection.execute(text("""\
CREATE TABLE meet_program_db.events
(
    event_number INT PRIMARY KEY,
    distance INT NOT NULL,
    gender VARCHAR(5) NOT NULL,
    stroke VARCHAR(45) NOT NULL,
    is_relay boolean NOT NULL,
    age_group VARCHAR(45) NOT NULL,
    CHECK(event_number >= 1 AND event_number <= 52),
    CHECK(distance IN(25, 50, 100, 200)),
    CHECK(gender IN('Boys', 'Girls')),
    CHECK(stroke IN('Freestyle', 'Backstroke', 'Breaststroke', 'Butterfly', 'Freestyle Relay', 'Medley Relay', 'Freestyle Relay Mixed Age')),
    #Should include 'Freestyle Relay Mixed Age' ???
    CHECK(age_group IN('8 & Under', '9-10', '11-12', '13-14', '15-18', '18 & Under'))
);"""))

sql_connection.execute(text("""\
CREATE TABLE meet_program_db.teams
(
	  team_abbreviation VARCHAR(5) PRIMARY KEY,
    team_name VARCHAR(45) NOT NULL
);"""))

sql_connection.execute(text("""\
CREATE TABLE meet_program_db.swimmers
(
	  swimmer_name VARCHAR(100) PRIMARY KEY,
    team_abbreviation VARCHAR(5) NOT NULL,
    gender VARCHAR(5) NOT NULL,
    # gender here needs to match formatting in Event table
    # there is also no identifier for gender in the meet program besides the event
    age INT NOT NULL,
    CHECK(gender IN('Boys', 'Girls')),
    CHECK(age>=5 AND age<=18),
    FOREIGN KEY (team_abbreviation) REFERENCES meet_program_db.teams(team_abbreviation) ON DELETE CASCADE
);"""))


sql_connection.execute(text("""\
CREATE TABLE meet_program_db.meets
(
    meet_date DATE PRIMARY KEY,
    meet_name VARCHAR(250) NOT NULL
);"""))

sql_connection.execute(text("""\
CREATE TABLE meet_program_db.meet_team
(
	  meet_team_id INT PRIMARY KEY AUTO_INCREMENT,
    meet_date DATE NOT NULL,
    team_abbreviation VARCHAR(5) NOT NULL,
    is_home BOOL NOT NULL,
    FOREIGN KEY (meet_date) REFERENCES meet_program_db.meets(meet_date) ON DELETE CASCADE,
    FOREIGN KEY (team_abbreviation) REFERENCES meet_program_db.teams(team_abbreviation) ON DELETE CASCADE
);"""))

sql_connection.execute(text("""\
CREATE TABLE meet_program_db.entries
(
  	entry_id INT PRIMARY KEY AUTO_INCREMENT,
  	event_number INT NOT NULL,
    meet_date DATE NOT NULL,
    seed_time TIME(2) NOT NULL,
    lane INT NOT NULL,
    swimmer_name VARCHAR(100) NOT NULL,
    expected_points REAL DEFAULT 0.0,
    expected_place INT,
    CHECK(lane>=1 AND lane<=6),
    CHECK (seed_time <'00:03:00'),
    FOREIGN KEY(event_number) REFERENCES meet_program_db.events(event_number) ON DELETE CASCADE,
    FOREIGN KEY(meet_date) REFERENCES meet_program_db.meets(meet_date) ON DELETE CASCADE,
    FOREIGN KEY(swimmer_name) REFERENCES meet_program_db.swimmers(swimmer_name) ON DELETE CASCADE
);"""))

#Uploading the PDF

In [ ]:
print("Please upload the PDF file:")
uploaded = files.upload()
file_name = next(iter(uploaded))
pdf_file = next(iter(uploaded.keys()))
pdf_content = uploaded[file_name]
print(f"Processing {file_name}...")

Please upload the PDF file:


Saving 2024-07-13 A-Meet CB@DR Meet Sheet with Times (1).pdf to 2024-07-13 A-Meet CB@DR Meet Sheet with Times (1).pdf
Processing 2024-07-13 A-Meet CB@DR Meet Sheet with Times (1).pdf...


# Extracting data from PDF into dataframes



## First Extraction

In [ ]:
def extract_meet_data(full_text):
    """Extract meet information and return as a DataFrame"""
    meets_data = []

    # Pattern 1: Matches "2024-06-29 A-Meet T@CB - 6/29/2024"
    pattern1 = r'(\d{4}-\d{2}-\d{2})\s+A-Meet\s+([^\n]+?)\s+-\s+\d{1,2}/\d{1,2}/\d{4}'

    # Pattern 2: Matches "NVSL A-Meet CB@DR - 7/13/2024"
    pattern2 = r'NVSL\s+A-Meet\s+([^\s]+)\s+-\s+(\d{1,2}/\d{1,2}/\d{4})'

    meet_date = 'Unknown'
    meet_name = 'Unknown'

    match1 = re.search(pattern1, full_text)
    match2 = re.search(pattern2, full_text)

    if match1:
        meet_date = match1.group(1)
        meet_name = match1.group(2)
    elif match2:
        meet_name = match2.group(1)
        meet_date = match2.group(2)  # in this case, the date comes second
        meet_date = dt.datetime.strptime(meet_date, '%m/%d/%Y').strftime('%Y-%m-%d')
    # Add meet to meets_data
    meets_data.append({
        'meet_date': meet_date,
        'meet_name': meet_name
    })

    return pd.DataFrame(meets_data)


def extract_team_data(full_text, meet_name):
    """Extract team abbreviations and names from swim meet PDF text."""
    teams_data = []

    # Step 1: Get team abbreviations from meet name
    away_home_pattern = r'([A-Z]+)@([A-Z]+)'
    team_match = re.search(away_home_pattern, meet_name)

    if not team_match:
        return pd.DataFrame(columns=['team_abbreviation', 'team_name'])

    away_abbrev, home_abbrev = team_match.group(1), team_match.group(2)

    # extract first individual event lanes 1 and 2
    individual_event_pattern = r'Event\s+1\s+.*?Heat\s+1.*?(\w+)\s+\d+.*?\n.*?(\w+)\s+\d+'
    individual_match = re.search(individual_event_pattern, full_text, re.DOTALL)

    if not individual_match:
        return pd.DataFrame(columns=['team_abbreviation', 'team_name'])

    first_lane_team = individual_match.group(1)
    second_lane_team = individual_match.group(2)

    # extract first relay event lanes 3 and 4
    relay_event_pattern = r'Event\s+41.*?Heat\s+1.*?A\s+[\d:.]+\s*([A-Za-z\s]+)\s.*?\n.*?A\s+[\d:.]+\s*([A-Za-z\s]+)\s'
    relay_match = re.search(relay_event_pattern, full_text, re.DOTALL)

    if not relay_match:
      return pd.DataFrame(columns=['team_abbreviation', 'team_name'])

    third_lane_team = relay_match.group(1).strip()
    fourth_lane_team = relay_match.group(2).strip()
    print("Team 1:", third_lane_team, "Team 2:", fourth_lane_team)

    # match using logic
    team_name_map = {
        home_abbrev: third_lane_team,
        away_abbrev: fourth_lane_team,
    }

    # Step 5: Create final DataFrame
    for abbrev in [away_abbrev, home_abbrev]:
        teams_data.append({
            'team_abbreviation': abbrev,
            'team_name': team_name_map.get(abbrev, abbrev)
        })

    return pd.DataFrame(teams_data)




def extract_meet_teams_data(full_text, meet_date, meet_name):
    """Extract meet teams relationship and return as a DataFrame"""
    meet_teams_data = []

    # Extract team abbreviations from entries
    entry_pattern = r'(\w+)\s+([\d\.]+)\s+(\d+)\s+([^_]+)\s+_+\s+(\d+)'
    team_abbrevs = set()
    for match in re.finditer(entry_pattern, full_text):
        team_abbrev = match.group(1)
        team_abbrevs.add(team_abbrev)

    # Extract home team abbreviation
    team_abbrev_pattern = r'@([A-Z]+)'  # Extract home team abbreviation after '@'
    home_team_abbrev = re.search(team_abbrev_pattern, meet_name)
    home_team_abbrev = home_team_abbrev.group(1) if home_team_abbrev else None

    # Create meet_teams_data
    for abbrev in team_abbrevs:
        meet_teams_data.append({
            'meet_date': meet_date,
            'team_abbreviation': abbrev,
            'is_home': abbrev == home_team_abbrev
        })

    return pd.DataFrame(meet_teams_data)

def extract_swimmers_data():
  parsed = parser.from_file('/content/'+str(pdf_file))
  heats = parsed['content'].split('Event')[1:]
  heat_lanes = []
  heat_info = []
  for heat in heats:
    heat_data = heat.split('Finals\n\n')
    heat_info.append(heat_data[0].split('\n\n'))
    heat_entries = heat_data[1].split('\n\n') #[entry for entry in  if entry != '']
    heat_lanes.append(heat_entries)

  columns = {"swimmer_name":[], "team_abbreviation":[], "gender":[], "age":[]}
  data = pd.DataFrame(columns)
  gender_map = {0:'Boys', 1:'Girls'}
  for i in range(40):
    gender = gender_map[i%2]
    heat = heat_lanes[i]
    for j in range(6):

      lane = heat[j]
      team = lane.split(' ')[0]
      lane_split = lane.split(' ')[2:-1]


      if '' in lane_split:
        lane_split.remove('')
      age_name = ''.join([' ' + i for i in lane_split]).strip()
      age = int(''.join([i for i in age_name if i.isdigit()]))
      name = ''.join([i for i in age_name if not i.isdigit()]).replace('  ', ' ')
      '''
      comma_ind = name.index(',')
      name = name[:comma_ind] + ', ' + name[comma_ind+1:]
      '''
      if name not in data['swimmer_name'].values:
        new_row = {"swimmer_name":[name], "team_abbreviation":[team], "gender":[gender], "age":[age]}
        data = pd.concat([data, pd.DataFrame(new_row)], ignore_index=True)

  return data


def extract_pdf_text(pdf_content):
    """Extract all text from a PDF and return as a string"""
    reader = PdfReader(io.BytesIO(pdf_content))
    full_text = "\n".join([page.extract_text() for page in reader.pages])
    return full_text

def process_swim_meet_pdf(pdf_content):
    """Process a swim meet PDF and return the requested dataframes"""
    # Extract all text from PDF
    full_text = extract_pdf_text(pdf_content)

    # Get meet information first ( need meet_date and meet_name for other functions)
    meets_df = extract_meet_data(full_text)
    meet_date = meets_df['meet_date'].iloc[0] if not meets_df.empty else 'Unknown'
    meet_name = meets_df['meet_name'].iloc[0] if not meets_df.empty else 'Unknown'

    # Extract other tables
    teams_df = extract_team_data(full_text, meet_name)
    meet_teams_df = extract_meet_teams_data(full_text, meet_date, meet_name)
    swimmers_df = extract_swimmers_data()

    return {
        'meets': meets_df,
        'teams': teams_df,
        'meet_teams': meet_teams_df,
        'swimmers': swimmers_df
    }

# upload the pdf file (this will trigger a file upload dialog)
'''print("Please upload your PDF file:")
uploaded = files.upload()
pdf_file = next(iter(uploaded.keys()))
'''
# Process the pdf and create data frames
#pdf_content = uploaded[pdf_file]
result_tables = process_swim_meet_pdf(pdf_content)

# display only the requested tables
print("\nSwimmers table:")
swimmers_df = result_tables['swimmers']
display(result_tables['swimmers'])

print("\nMeets table:")
display(result_tables['meets'])
meets_df = result_tables['meets']

print("\nTeams table:")
display(result_tables['teams'])
teams_df = result_tables['teams']

print("\nMeet_Teams table:")
display(result_tables['meet_teams'])
meet_teams_df = result_tables['meet_teams']

Team 1: Donaldson Run Team 2: Chesterbrook

Swimmers table:


,swimmer_name,team_abbreviation,gender,age
0,"Baca, Nicholas",DR,Boys,8.0
1,"Couchara, George",CB,Boys,7.0
2,"Jacobs, Christian",DR,Boys,7.0
3,"Landsman, Ezra",CB,Boys,8.0
4,"Simms, Hunter",DR,Boys,8.0
...,...,...,...,...
140,"Erdem, Ela",CB,Girls,9.0
141,"Mehmood, Ayla",DR,Girls,14.0
142,"Christofferson, Marie",CB,Girls,14.0
143,"Leichty, Hadley",DR,Boys,16.0



Meets table:


,meet_date,meet_name
0,2024-07-13,CB@DR



Teams table:


,team_abbreviation,team_name
0,CB,Chesterbrook
1,DR,Donaldson Run



Meet_Teams table:


,meet_date,team_abbreviation,is_home
0,2024-07-13,DR,True
1,2024-07-13,CB,False


## Extracting Events (Alden)

In [ ]:
def extract_event_info():
  parsed = parser.from_file('/content/'+str(pdf_file))
  heats = parsed['content'].split('Event')[1:]
  heat_lanes = []
  heat_info = []
  for heat in heats:
    heat_data = heat.split('Finals\n\n')
    heat_info.append(heat_data[0].split('\n\n'))
    heat_entries = heat_data[1].split('\n\n') #[entry for entry in  if entry != '']
    heat_lanes.append(heat_entries)

  event_nums = []
  distances = []
  genders = []
  strokes = []
  age_groups = []

  data = {"event_number":[], "distance":[], "gender":[], "stroke":[], "is_relay":[], "age_group":[]}
  event_data = pd.DataFrame(data)
  for heat in heat_info:
    heat_description = heat[0]
    heat_description.removeprefix(' ')
    stroke = heat_description.split(' SC Meter ')[-1]
    event_num = int(heat_description.split(' SC Meter ')[0].split('  ')[1])
    gender = heat_description.split(' SC Meter ')[0].split('  ')[2][:6].strip()
    distance = int(heat_description.split(' SC Meter ')[0].split('  ')[2][6:].strip()[-3:].strip())
    age_group = heat_description.split(' SC Meter ')[0].split('  ')[2][6:].strip()[:-3].strip()
    is_relay = bool(stroke.find('Relay')!=-1)
    new_row = {"event_number":[event_num], "distance":[distance], "gender":[gender], "stroke":[stroke], "is_relay":[is_relay], "age_group":[age_group]}
    event_data = pd.concat([event_data, pd.DataFrame(new_row)], ignore_index=True)
  return event_data

events_df = extract_event_info()
events_df

,event_number,distance,gender,stroke,is_relay,age_group
0,1.0,25.0,Boys,Freestyle,0.0,8 & Under
1,2.0,25.0,Girls,Freestyle,False,8 & Under
2,3.0,50.0,Boys,Freestyle,False,9-10
3,4.0,50.0,Girls,Freestyle,False,9-10
4,5.0,50.0,Boys,Freestyle,False,11-12
5,6.0,50.0,Girls,Freestyle,False,11-12
6,7.0,50.0,Boys,Freestyle,False,13-14
7,8.0,50.0,Girls,Freestyle,False,13-14
8,9.0,50.0,Boys,Freestyle,False,15-18
9,10.0,50.0,Girls,Freestyle,False,15-18


##Extracting Entries##

In [ ]:
 def format_seed_time_for_sql(time_str):
    """
    Converts a time string like MM:SS.xx or SS.xx into strict HH:MM:SS.MICROSECONDS format.
    """
    if pd.isnull(time_str) or time_str == '':
        return None

    try:
        if ':' in time_str:
            minutes, seconds = time_str.split(':')
            minutes = int(minutes)
            seconds = float(seconds)
        else:
            minutes = 0
            seconds = float(time_str)

        total_seconds = minutes * 60 + seconds

        hours = int(total_seconds // 3600)
        minutes = int((total_seconds % 3600) // 60)
        seconds_whole = int(total_seconds % 60)
        microseconds = int(round((total_seconds - int(total_seconds)) * 1_000_000))

        return f"{hours:02d}:{minutes:02d}:{seconds_whole:02d}.{microseconds:06d}"
    except Exception as e:
        print(f"Error parsing time: {time_str} -> {e}")
        return None

def extract_meet_date(text):
    # Try to capture MM/DD/YYYY after the “A‑Meet … - ” line
    m = re.search(r"A-Meet\b.*?-\s*(\d{1,2}/\d{1,2}/\d{4})", text)
    if m:
        date_str = m.group(1)
    else:
        # fallback: grab all YYYY‑MM‑DD or M/D/YYYY substrings
        pattern = r'\b(?:\d{4}-\d{2}-\d{2}|\d{1,2}/\d{1,2}/\d{4})\b'
        all_dates = re.findall(pattern, text)
        date_str = all_dates[-1] if all_dates else None

    if not date_str:
        raise ValueError("No date found in text")

    # parse into a datetime
    for fmt in ("%Y-%m-%d", "%m/%d/%Y"):
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue

    # if we get here, the string didn’t match either format
    raise ValueError(f"Unrecognized date format: {date_str}")

def extract_text(file_path):
    reader = PdfReader(io.BytesIO(file_path))
    return "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])

# define the main extraction function
def extract_all_swim_data(pdf_content):
    full_text = extract_text(pdf_content)
    meet_date = extract_meet_date(full_text)
def extract_entry_data(pdf_content):
    full_text = extract_text(pdf_content)
    meet_date = extract_meet_date(full_text)

    entries_data = []

    #find event headers
    event_header_pattern = r'Event\s+(\d+)\s+(Boys|Girls)\s+(.+?)\s+(\d+)\s+SC Meter\s+(.+?)\n'
    event_headers = list(re.finditer(event_header_pattern, full_text))

    for idx, header_match in enumerate(event_headers):
        event_num, gender, age_group, distance, stroke = header_match.groups()

        start_pos = header_match.end()
        end_pos = event_headers[idx+1].start() if idx+1 < len(event_headers) else len(full_text)
        event_block = full_text[start_pos:end_pos]

        if "relay" not in stroke.lower():
            #find swimmers inside event block
            swimmer_entry_pattern = r'(\w+)\s+([\d\.]+|NT)\s+(\d+)\s+(.+?)\s+(\d+)'
            swimmer_matches = re.findall(swimmer_entry_pattern, event_block)

            for team, seed_time, age, swimmer_name, lane in swimmer_matches:
                if seed_time == 'NT':
                    fixed_seed_time = '00:00:00.000000'  # Treat NT as 0 seconds
                else:
                    fixed_seed_time = seed_time.strip()

                entries_data.append({
                    'event_number': int(event_num),
                    'meet_date': meet_date,
                    'seed_time': fixed_seed_time,
                    'lane': int(lane),
                    'swimmer_name': swimmer_name.replace('_', '').strip()

    })

    #extract relay events
    event_blocks = re.split(r'(Event\s+\d+\s+)', full_text)

    for i in range(1, len(event_blocks) - 1, 2):
        header = event_blocks[i].strip()
        content = event_blocks[i+1]

        event_number_match = re.search(r'Event\s+(\d+)', header)
        if not event_number_match:
            continue

        event_number = int(event_number_match.group(1))

        if 41 <= event_number <= 52:
            lines = content.strip().split('\n')
            team_counter = 0
            j = 0

            while j < len(lines):
                line = lines[j].strip()

                team_match = re.match(r'\s*(?:A\s*)?(\d{1,2}(?::\d{2})?\.\d{2})\s+([A-Za-z].+)', line)
                if team_match:
                    seed_time, team_name = team_match.groups()
                    seed_time = seed_time.strip()
                    swimmer_names = []

                    for k in range(1, 10):
                        if j + k < len(lines):
                            swimmer_line = lines[j + k].strip()

                            if re.match(r'\s*(?:A\s*)?(\d{1,2}(?::\d{2})?\.\d{2})\s+[A-Za-z]', swimmer_line):
                                break

                            swimmer_matches = re.findall(r'\d\)\s*(.*?)\s+\d{1,2}', swimmer_line)
                            for name in swimmer_matches:
                                swimmer_names.append(name.strip())

                            if len(swimmer_names) == 4:
                                break

                    if len(swimmer_names) == 4:
                        lane = 3 if team_counter == 0 else 4

                        for swimmer in swimmer_names:
                            entries_data.append({
                                'event_number': event_number,
                                'meet_date': meet_date,
                                'seed_time': seed_time,
                                'lane': lane,
                                'swimmer_name': swimmer
                            })
                        team_counter += 1
                    j += k
                else:
                    j += 1

    #build final df
    final_df = pd.DataFrame(entries_data)
    final_df = final_df[['event_number', 'meet_date', 'seed_time', 'lane', 'swimmer_name']]
    final_df['event_number'] = final_df['event_number'].astype(int)
    final_df['lane'] = final_df['lane'].astype(int)
    final_df['seed_time'] = final_df['seed_time'].apply(format_seed_time_for_sql)

    return final_df

entries_df = extract_entry_data(pdf_content)
entries_df

,event_number,meet_date,seed_time,lane,swimmer_name
0,1,2024-07-13,00:00:21.310000,1,"Baca, Nicholas"
1,1,2024-07-13,00:00:19.340000,2,"Couchara, George"
2,1,2024-07-13,00:00:18.810000,3,"Jacobs, Christian"
3,1,2024-07-13,00:00:17.590000,4,"Landsman, Ezra"
4,1,2024-07-13,00:00:19.970000,5,"Simms, Hunter"
...,...,...,...,...,...
331,52,2024-07-13,00:02:02.320000,3,"Conley, Rachel"
332,52,2024-07-13,00:01:59.870000,4,"Wilson, Mara"
333,52,2024-07-13,00:01:59.870000,4,"Wilson, Lily"
334,52,2024-07-13,00:01:59.870000,4,"Cory, McKenzie"


# Importing Extracted Data Into SQL Database

In [ ]:
#upload unique values extracted from the pdf to the SQL database

#events
events_df[~events_df['event_number'].isin(pd.read_sql('SELECT event_number FROM meet_program_db.events', sqlEngine)['event_number'])].to_sql(con = sqlEngine, schema = 'meet_program_db', name = 'events', if_exists = 'append', index = False)

#teams
teams_df[~teams_df['team_abbreviation'].isin(pd.read_sql('SELECT team_abbreviation FROM meet_program_db.teams', sqlEngine)['team_abbreviation'])].to_sql(con = sqlEngine, schema = 'meet_program_db', name = 'teams', if_exists = 'append', index = False)

#swimmers
swimmers_df[~swimmers_df['swimmer_name'].isin(pd.read_sql('SELECT swimmer_name FROM meet_program_db.swimmers', sqlEngine)['swimmer_name'])].to_sql(con = sqlEngine, schema = 'meet_program_db', name = 'swimmers', if_exists = 'append', index = False)

#meets
meets_df[~meets_df['meet_date'].isin(pd.read_sql('SELECT meet_date FROM meet_program_db.meets', sqlEngine)['meet_date'])].to_sql(con = sqlEngine, schema = 'meet_program_db', name = 'meets', if_exists = 'append', index = False)

#meet_teams
meet_teams_df[~meet_teams_df['meet_date'].isin(pd.read_sql('SELECT meet_date FROM meet_program_db.meet_team', sqlEngine)['meet_date'])].to_sql(con = sqlEngine, schema = 'meet_program_db', name = 'meet_team', if_exists = 'append', index = False)

#clean the entries dataframe so that any entries with "NT" as their seed time are replaced with the minimum for that event.
for i in range(len(entries_df.index)):
  if(entries_df['seed_time'].loc[i] == None):
    entries_df.at[i, 'seed_time'] = entries_df['seed_time'].iloc[i-1]

#entries
entries_df[(~entries_df['meet_date'].isin(pd.read_sql('SELECT meet_date FROM meet_program_db.entries', sqlEngine)['meet_date']) & (entries_df['swimmer_name'].isin(pd.read_sql('SELECT swimmer_name FROM meet_program_db.swimmers',sqlEngine)['swimmer_name'])))].to_sql(con = sqlEngine, schema = 'meet_program_db', name = 'entries', if_exists = 'append', index = False)



<ipython-input-18-03f9e30e48d3>:24: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  entries_df[(~entries_df['meet_date'].isin(pd.read_sql('SELECT meet_date FROM meet_program_db.entries', sqlEngine)['meet_date']) & (entries_df['swimmer_name'].isin(pd.read_sql('SELECT swimmer_name FROM meet_program_db.swimmers',sqlEngine)['swimmer_name'])))].to_sql(con = sqlEngine, schema = 'meet_program_db', name = 'entries', if_exists = 'append', index = False)


334

# Colin's Code

In [ ]:
def swing_score_calculator (time_to_subtract, meet_date):
  swing_score = pd.DataFrame(sql_connection.execute(text("""\
    SELECT entries.entry_id, entries.meet_date, entries.event_number, is_relay, lane, entries.swimmer_name, swimmers.team_abbreviation, is_home, seed_time
    FROM meet_program_db.entries
    LEFT JOIN meet_program_db.swimmers ON entries.swimmer_name = swimmers.swimmer_name
    LEFT JOIN meet_program_db.events ON entries.event_number = events.event_number
    LEFT JOIN meet_program_db.teams ON swimmers.team_abbreviation = teams.team_abbreviation
    LEFT JOIN meet_program_db.meet_team ON teams.team_abbreviation = meet_team.team_abbreviation AND entries.meet_date = meet_team.meet_date
    WHERE entries.meet_date = '""" + str(meet_date) + """\'
    ORDER BY event_number, seed_time, event_number;
  """)))
  # if time_to_subtract is positive, subtract the given time from the home team
  if(time_to_subtract > 0):
    for i in range(len(swing_score.index)):
      if(swing_score['is_home'].iloc[i] == 1):
        current_time = dt.datetime(1,1,1,0,int(swing_score['seed_time'].iloc[i].seconds/60), swing_score['seed_time'].iloc[i].seconds%60, int(float(swing_score['seed_time'].iloc[i].microseconds)))
        dropped_time = dt.datetime(1,1,1,0,0,int(abs(time_to_subtract)),abs(int(round(abs(time_to_subtract)%1,2)*1000000)))
        new_time = current_time-dropped_time
        swing_score.at[i, 'seed_time'] = new_time
  # if time_to_subtract is negative, subract the given time from the away team
  elif(time_to_subtract < 0):
    for i in range(len(swing_score.index)):
      if(swing_score['is_home'].iloc[i] == 0):
        current_time = dt.datetime(1,1,1,0,int(swing_score['seed_time'].iloc[i].seconds/60), swing_score['seed_time'].iloc[i].seconds%60, int(float(swing_score['seed_time'].iloc[i].microseconds)))
        dropped_time = dt.datetime(1,1,1,0,0,int(abs(time_to_subtract)),abs(int(round(abs(time_to_subtract)%1,2)*1000000)))
        new_time = current_time-dropped_time
        swing_score.at[i, 'seed_time'] = new_time  # if time_to_subtract is zero, do nothing
  swing_score['points'] = 0.0
  swing_score.sort_values(by=['event_number', 'seed_time'], ignore_index = True,inplace = True)

  i = 0
  # For individual events
  while (i+2 < len(swing_score.index)) & (swing_score['is_relay'].iloc[i] == 0):
    #check for a tie for first
    if(swing_score['seed_time'].iloc[i] == swing_score['seed_time'].iloc[i+1]):
      #check for three way tie
      if(swing_score['seed_time'].iloc[i] == swing_score['seed_time'].iloc[i+2]):
        swing_score.at[i, 'points'] = 3.0
        swing_score.at[i + 1, 'points'] = 3.0
        swing_score.at[i + 2, 'points'] = 3.0
      else:
        swing_score.at[i, 'points'] = 4.0
        swing_score.at[i+1, 'points'] = 4.0
        #check for a tie for third
        if(swing_score['seed_time'].iloc[i+2] == swing_score['seed_time'].iloc[i+3]):
          #check for three way tie for third
          if(swing_score['seed_time'].iloc[i+2] == swing_score['seed_time'].iloc[i+4]):
            swing_score.at[i+2, 'points'] = 1.0/3
            swing_score.at[i+3, 'points'] = 1.0/3
            swing_score.at[i+4, 'points'] = 1.0/3
          else:
            swing_score.at[i+2, 'points'] = 1.0/2
            swing_score.at[i+3, 'points'] = 1.0/2
        else:
          swing_score.at[i+2, 'points'] = 1.0
    else:
      swing_score.at[i, 'points'] = 5.0
      #check for a tie for second
      if(swing_score['seed_time'].iloc[i+1] == swing_score['seed_time'].iloc[i+2]):
        #check for a three way tie for second
        if(swing_score['seed_time'].iloc[i+1] == swing_score['seed_time'].iloc[i+3]):
          swing_score.at[i+1, 'points'] = 4.0/3
          swing_score.at[i+2, 'points'] = 4.0/3
          swing_score.at[i+3, 'points'] = 4.0/3
        else:
          swing_score.at[i+1, 'points'] = 2.0
          swing_score.at[i+2, 'points'] = 2.0
      else:
        swing_score.at[i+1, 'points'] = 3.0
        #check for a tie for third
        if(swing_score['seed_time'].iloc[i+2] == swing_score['seed_time'].iloc[i+3]):
          #check for a three way tie for third
          if(swing_score['seed_time'].iloc[i+2] == swing_score['seed_time'].iloc[i+4]):
            swing_score.at[i+2, 'points'] = 1.0/3
            swing_score.at[i+3, 'points'] = 1.0/3
            swing_score.at[i+4, 'points'] = 1.0/3
          else:
            swing_score.at[i+2, 'points'] = 1.0/2.0
            swing_score.at[i+3, 'points'] = 1.0/2
        else:
          swing_score.at[i+2, 'points'] = 1.0
    while (i < len(swing_score.index)-2) & (swing_score['event_number'].iloc[i] == swing_score['event_number'].iloc[i+1]):
      i += 1
    i+=1
  #For Relay Events
  while (i< len(swing_score.index)):
    #Check to see if relays tied
    if(swing_score['seed_time'].iloc[i] == swing_score['seed_time'].iloc[i+4]):
      for k in range (8):
        swing_score.at[i+k, 'points'] = 5.0/8
    else:
      for k in range (4):
        swing_score.at[i+k,'points'] = 5.0/4
    i  += 8

  home_team_points = 0
  away_team_points = 0
  for i in range (len(swing_score.index)):
    if(swing_score['is_home'].iloc[i] == 1):
      home_team_points += swing_score['points'].iloc[i]
    else:
      away_team_points += swing_score['points'].iloc[i]
  return{
      'df' : swing_score,
      'home_team_points' : home_team_points,
      'away_team_points' : away_team_points,
  }

In [ ]:
score_possibilities = pd.DataFrame(columns=['meet_date', 'time_to_subtract', 'home_team_score', 'away_team_score'])
index = 0
# create a row in the score_possibilities table for every time_to_subtract and every meet
for meet_date in (pd.DataFrame(sql_connection.execute(text("SELECT meet_date FROM meet_program_db.meets;")))['meet_date']):
  for i in range (-10, 11):
    dict = swing_score_calculator(float(i/10), meet_date)
    score_possibilities.loc[index] = [meet_date, float(i/10), dict['home_team_points'], dict['away_team_points']]
    # if the time_to_subtract is equal to zero, update the SQL database expected_points column
    if(i==0):
      df = dict['df']
      for k in range(len(df.index)):
        if(df['points'].iloc[k] != 0):
          string_to_execute = 'UPDATE meet_program_db.entries SET expected_points = ' + str(df['points'].iloc[k]) + ' WHERE entry_id = ' + str(df['entry_id'].iloc[k]) + ';'
          sql_connection.execute(text(string_to_execute))
    index += 1
# go through the score possibilities table and convert the meet_date to a string
for i in range(len(score_possibilities.index)):
  score_possibilities.at[i, 'meet_date'] = dt.datetime.strftime(score_possibilities['meet_date'].iloc[i],'%m/%d/%Y')
score_possibilities

,meet_date,time_to_subtract,home_team_score,away_team_score
0,06/22/2024,-1.0,215.0,205.0
1,06/22/2024,-0.9,219.0,201.0
2,06/22/2024,-0.8,221.0,199.0
3,06/22/2024,-0.7,221.0,199.0
4,06/22/2024,-0.6,224.0,196.0
...,...,...,...,...
58,07/13/2024,0.6,170.5,249.5
59,07/13/2024,0.7,173.5,246.5
60,07/13/2024,0.8,176.5,243.5
61,07/13/2024,0.9,179.0,241.0


# Ciaran's Question

In [ ]:
def find_top_scorers(meet_date, gender="both", team_abbreviation="both", age_group="all"):
  events = pd.DataFrame(sql_connection.execute(text('SELECT * FROM meet_program_db.events')))
  swimmers = pd.DataFrame(sql_connection.execute(text('SELECT * FROM meet_program_db.swimmers')))
  point_table = swing_score_calculator(0, meet_date)['df']
  # First join: bring in AgeGroup based on EventNumber
  point_table = point_table.merge(events[['event_number', 'age_group']], on='event_number', how='left')
  # Second join: bring in Gender based on SwimmerName
  point_table = point_table.merge(swimmers[['swimmer_name', 'gender']], on='swimmer_name', how='left')
  if gender != "both":
    if gender.lower() in ["male", "m"]:
      gender = "Boys"
    if gender.lower() in ["female", "f"]:
      gender = "Girls"
    point_table = point_table[point_table['gender'] == gender]
  if team_abbreviation != "both":
    point_table = point_table[point_table['team_abbreviation'] == team_abbreviation]
  if age_group != "all":
    point_table = point_table[point_table['age_group'] == age_group]
  swimmer_totals = point_table.groupby(['swimmer_name', 'team_abbreviation', 'age_group', 'gender'])['points'].sum().reset_index()

  top_scorers = swimmer_totals.sort_values('points', ascending=False).head(5).reset_index(drop=True)
  top_scorers.insert(0, 'Rank', range(1, len(top_scorers) + 1))


  return top_scorers


find_top_scorers('2024-06-22')


,Rank,swimmer_name,team_abbreviation,age_group,gender,points
0,1,"Bhargava, Arav",CB,15-18,Boys,11.25
1,2,"Cory, Maddox",CB,11-12,Boys,11.25
2,3,"Cory, McKenzie",CB,13-14,Girls,11.25
3,4,"Maher, Manu",CB,13-14,Boys,11.25
4,5,"Maher, Emma",CB,9-10,Girls,11.25


# Alden's Code

In [ ]:
def add_expected_places():
  entries = pd.DataFrame(sql_connection.execute(text("SELECT * FROM meet_program_db.entries;")))
  entries_sorted = entries.sort_values(by=['event_number', 'meet_date', 'seed_time'])
  current_event = None
  current_meet = None
  place_counter = 0
  current_place = 0

  for index, row in entries_sorted.iterrows():
    if current_event != row['event_number'] or current_meet != row['meet_date']:
      # Reset place counter for a new event or meet
      current_event = row['event_number']
      current_meet = row['meet_date']
      current_place = 1
      previous_seed_time = row['seed_time']
      place_increment = 0
    else:
      if row['seed_time'] == previous_seed_time:
        # If the seed time is the same as the previous row, it's a tie
        if row['event_number'] < 41:
          # Increment if it isn't a relay to account for tied place
          place_increment += 1
      else:
        # Otherwise, increment the place counter
        current_place += place_increment + 1
        place_increment = 0

    # Update the expected_place in the entries DataFrame
    entries.at[index, 'expected_place'] = current_place
    previous_seed_time = row['seed_time']
    string_to_execute = 'UPDATE meet_program_db.entries SET expected_place = ' + str(current_place) + ' WHERE entry_id = ' + str(row['entry_id']) + ';'
    sql_connection.execute(text(string_to_execute))
  return entries

entries_with_places = add_expected_places()

display(pd.DataFrame(sql_connection.execute(text("SELECT * FROM meet_program_db.entries;"))))

,entry_id,event_number,meet_date,seed_time,lane,swimmer_name,expected_points,expected_place
0,1,1,2024-06-29,0 days 00:00:20.330000,1,"King, Ethan",0.00,6
1,2,1,2024-06-29,0 days 00:00:19.210000,2,"Kain, Abe",1.00,3
2,3,1,2024-06-29,0 days 00:00:18.390000,3,"Landsman, Ezra",4.00,1
3,4,1,2024-06-29,0 days 00:00:18.390000,4,"Laudier, Liam",4.00,1
4,5,1,2024-06-29,0 days 00:00:19.340000,5,"Couchara, George",0.00,4
...,...,...,...,...,...,...,...,...
1000,1001,52,2024-07-13,0 days 00:02:02.320000,3,"Conley, Rachel",0.00,2
1001,1002,52,2024-07-13,0 days 00:01:59.870000,4,"Wilson, Mara",0.00,1
1002,1003,52,2024-07-13,0 days 00:01:59.870000,4,"Wilson, Lily",0.00,1
1003,1004,52,2024-07-13,0 days 00:01:59.870000,4,"Cory, McKenzie",1.25,1


In [ ]:
def get_time_diffs():
  query = f"""
  SELECT
      ev.stroke,
      ev.distance,
      ev.gender,
      e1.event_number,
      e1.meet_date,
      ABS(e1.seed_time - e2.seed_time) AS time_difference,
      e1.lane AS lane1,
      e2.lane AS lane2,
      e1.expected_place AS place1,
      e2.expected_place AS place2
  FROM
      meet_program_db.entries AS e1
  JOIN
      meet_program_db.entries AS e2
  ON
      e1.event_number = e2.event_number
      AND e1.meet_date = e2.meet_date
  JOIN meet_program_db.events AS ev
  ON e1.event_number = ev.event_number
  WHERE
      (e1.expected_place < e2.expected_place
      OR (e1.expected_place = e2.expected_place AND e1.lane < e2.lane))
      AND e1.expected_place <= 3
  ORDER BY time_difference ASC;
  """
  # Put the results into a DataFrame
  time_diff_df = pd.DataFrame(sql_connection.execute(text(query)))
  return time_diff_df

time_diff_df = get_time_diffs()
time_diff_df

,stroke,distance,gender,event_number,meet_date,time_difference,lane1,lane2,place1,place2
0,Freestyle,25,Boys,1,2024-06-29,0.00,3,4,1,1
1,Breaststroke,25,Girls,22,2024-06-29,0.00,5,6,2,2
2,Butterfly,25,Boys,31,2024-06-29,0.00,5,6,2,2
3,Backstroke,50,Girls,16,2024-07-13,0.00,2,3,2,2
4,Freestyle,50,Girls,8,2024-06-29,0.01,4,5,2,3
...,...,...,...,...,...,...,...,...,...,...
2000,Medley Relay,100,Boys,47,2024-06-22,47.34,3,4,1,2
2001,Medley Relay,100,Boys,47,2024-06-22,47.34,3,4,1,2
2002,Medley Relay,100,Boys,47,2024-06-22,47.34,3,4,1,2
2003,Medley Relay,100,Boys,47,2024-06-22,47.34,3,4,1,2


# Will's Code

In [ ]:
#analysis
import pandas as pd
from sqlalchemy import text

# mappings
AGE_GROUP_MAP = {
    **{i: "8 & Under" for i in (1,2,11,12,21,22,31,32)},
    **{i: "9-10"      for i in (3,4,13,14,23,24,33,34)},
    **{i: "11-12"     for i in (5,6,15,16,25,26,35,36)},
    **{i: "13-14"     for i in (7,8,17,18,27,28,37,38)},
    **{i: "15-18"     for i in (9,10,19,20,29,30,39,40)},
}
STROKE_MAP = {
    **dict.fromkeys(range(1,11),   "Freestyle"),
    **dict.fromkeys(range(11,21),  "Backstroke"),
    **dict.fromkeys(range(21,31),  "Breaststroke"),
    **dict.fromkeys(range(31,41),  "Butterfly"),
}

STROKES    = ["Backstroke","Breaststroke","Butterfly","Freestyle"]
AGE_GROUPS = ["9-10","11-12","13-14","15-18","Mixed Age"]

def load_data(sql_connection, meet_date="2024-06-29"):
    """Load entries, map age/stroke/gender, drop 8 & Under."""
    sql = text("""
        SELECT
            e.event_number,
            e.swimmer_name,
            s.team_abbreviation,
            e.seed_time
        FROM meet_program_db.entries AS e
        LEFT JOIN meet_program_db.swimmers AS s
          ON e.swimmer_name = s.swimmer_name
        WHERE e.meet_date = :md
          AND e.event_number BETWEEN 1 AND 40
        ORDER BY e.event_number, e.seed_time;
    """)
    df = pd.DataFrame(
        sql_connection.execute(sql, {"md": meet_date}).fetchall(),
        columns=["event_number","swimmer_name","team_abbreviation","seed_time"]
    )
    df["seed_time_sec"] = df["seed_time"].apply(lambda td: td.total_seconds())
    df["age_group"] = df["event_number"].map(AGE_GROUP_MAP)
    df["stroke"]    = df["event_number"].map(STROKE_MAP)
    df["gender"]    = df["event_number"].apply(lambda x: "Male" if x % 2 else "Female")
    return df[df["age_group"] != "8 & Under"] # gets rid of 8 & unders because not enough of them racing individually to build a relay with

def get_relay_combos(df, age_group):
    """Return list of {'label', 'leg_swimmers', 'leg_times', 'total'}."""
    teams = sorted(df["team_abbreviation"].unique())
    combos = []

    if age_group != "Mixed Age":
        df_age = df[df["age_group"] == age_group]
        for team in teams:
            for gender in ("Male","Female"):
                grp = df_age[(df_age.team_abbreviation==team)&(df_age.gender==gender)]
                used, times, names, total = set(), [], [], 0.0
                for stroke in STROKES:
                    cands = grp[grp.stroke==stroke].sort_values("seed_time_sec")
                    for _, r in cands.iterrows():
                        nm, t = r.swimmer_name, r.seed_time_sec
                        if nm not in used:
                            # split time in these age groups
                            if age_group=="9-10" and stroke!="Butterfly":
                                t /= 2
                            elif age_group in ("11-12","13-14"):
                                t /= 2
                            used.add(nm)
                            names.append(nm)
                            times.append(t)
                            total += t
                            break
                if len(times) == 4:
                    combos.append({
                        "label": f"{team} {gender}",
                        "leg_swimmers": names,
                        "leg_times":   times,
                        "total":       total
                    })

    else:
        # mixed age - fastest 4 freestyle splits per team/gender
        for team in teams:
            for gender in ("Male","Female"):
                grp = df[(df.stroke=="Freestyle") & (df.gender==gender)
                         & (df.team_abbreviation==team)]
                used, times, names, total = set(), [], [], 0.0
                for _, r in grp.sort_values("seed_time_sec").iterrows():
                    nm, t = r.swimmer_name, r.seed_time_sec/2
                    if nm not in used:
                        used.add(nm)
                        names.append(nm)
                        times.append(t)
                        total += t
                    if len(times) == 4:
                        combos.append({
                            "label": f"{team} {gender}",
                            "leg_swimmers": names,
                            "leg_times":   times,
                            "total":       total
                        })
                        break

    combos.sort(key=lambda c: (c["label"].split()[-1], c["label"].split()[0]))
    return combos


# Dashboard

In [ ]:
meet_list = [dt.datetime.strptime(date, '%m/%d/%Y').date() for date in score_possibilities['meet_date'].unique()]

app = Dash(__name__)
app.layout = html.Div([
    html.Div([
        html.H2("Meet"),
        dcc.Dropdown(meet_list, meet_list[0], id='meet_dropdown'),
    ]),

    html.Div([
        html.H2("Top 5 Scorers (Filtered)"),
        html.Label("Gender"),
        dcc.Dropdown(
            options=[
                {"label": "Both", "value": "both"},
                {"label": "Boys", "value": "male"},
                {"label": "Girls", "value": "female"}
            ],
            value="both",
            id="gender_dropdown"
        ),
        html.Label("Team"),
        dcc.Dropdown(
            options=[{"label": team, "value": team} for team in pd.DataFrame(sql_connection.execute(text('SELECT * FROM meet_program_db.teams')))['team_abbreviation'].unique()] + [{"label": "Both", "value": "both"}],
            value="both",
            id="team_dropdown"
        ),
        html.Label("Age Group"),
        dcc.Dropdown(
            options=[
        {'label': '8 & Under', 'value': '8 & Under'},
        {'label': '9-10', 'value': '9-10'},
        {'label': '11-12', 'value': '11-12'},
        {'label': '13-14', 'value': '13-14'},
        {'label': '15-18', 'value': '15-18'},
        {'label': 'All', 'value': 'all'}
            ],
            value="all",
            id="age_dropdown"
        ),
        dcc.Graph(id='top_5_scorers_graph')
    ], style = {'width':'49%', 'display':'inline-block'}),

    html.Div([
        html.H2("Score given a ___ second swing in time"),
        dcc.Graph(id='swing_graph'),
        dcc.Slider(-1, 1, step=.1, value=0, id='swing_slider', tooltip={"always_visible": True, "template": "{value} second swing"})

    ], style= {'width':'49%', 'display':'inline-block', 'float':'right'}),

    html.Div([
        html.H2("Closest Events"),
        dcc.Graph(id='close_events_graph'),
    ])
])

@callback(
    Output('swing_graph', 'figure'),
    Input('swing_slider', 'value'),
    Input('meet_dropdown', 'value')
)
def update_swing_graph(swing_slider, meet_dropdown):
  #update data
  colors = ['blue', 'red', 'blue', 'red']
  dff = score_possibilities[pd.to_datetime(score_possibilities['meet_date'], format='%m/%d/%Y') == dt.datetime.strptime(meet_dropdown, '%Y-%m-%d')]
  fig = px.bar(x=['Away Original','Away Swing', 'Home Original', 'Home Swing'], y=[dff[dff['time_to_subtract'] == 0]['away_team_score'].iloc[0], dff[dff['time_to_subtract'] == swing_slider]['away_team_score'].iloc[0], dff[dff['time_to_subtract'] == 0]['home_team_score'].iloc[0], dff[dff['time_to_subtract'] == swing_slider]['home_team_score'].iloc[0]])
  fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, yaxis={'range':[0,350], 'title':{'text':'Points'}}, xaxis={'title':{'text':'Time Swing'}})
  fig.update_traces(marker_color=colors)
  return fig

@callback(
    Output('close_events_graph', 'figure'),
    Input('meet_dropdown', 'value')
)
def update_close_events_graph(meet_dropdown):
    temp_time_diff_df = time_diff_df.copy(deep=True)
    time_diffs = temp_time_diff_df[temp_time_diff_df['meet_date'] == dt.datetime.strptime(meet_dropdown, '%Y-%m-%d').date()]
    top_10_closest = time_diffs.sort_values(by='time_difference').head(10)
    top_10_closest['place_label'] = 'Event ' + top_10_closest['event_number'].astype(str) + ': Place ' + top_10_closest['place1'].astype(str) + ' to ' + top_10_closest['place2'].astype(str)
    dropdown_todate = dt.datetime.strptime(meet_dropdown, '%Y-%m-%d').date()
    fig = px.bar(top_10_closest, x='place_label', y='time_difference', title='Top 10 Closest Races')
    fig.update_layout(
        margin={'l': 40, 'b': 40, 't': 40, 'r': 0},
        yaxis_title='Time Difference (seconds)',
        xaxis_title='Expected Places',
        xaxis={'categoryorder':'total ascending'}
    )
    return fig

@callback(
    Output('top_5_scorers_graph', 'figure'),
    Input('meet_dropdown', 'value'),
    Input('gender_dropdown', 'value'),
    Input('team_dropdown', 'value'),
    Input('age_dropdown', 'value')
)
def update_top_5_scorers(meet_dropdown, gender, team, age):
    meet_date = str(dt.datetime.strptime(meet_dropdown, '%Y-%m-%d'))
    top_scorers_df = find_top_scorers(meet_date, gender, team, age)
    fig = px.bar(top_scorers_df,
                 x='swimmer_name',
                 y='points',
                 color='team_abbreviation',
                 text='points',
                 title='Top 5 Individual Scorers')
    fig.update_layout(
        xaxis_title='Swimmer Name',
        yaxis_title='Total Points',
        margin={'l': 40, 'b': 40, 't': 40, 'r': 0}
    )
    return fig

# run
if __name__ == '__main__':
    app.run(debug=False, jupyter_mode="inline")

<IPython.core.display.Javascript object>

# Closing the Connection

In [ ]:
# keep in mind to close the connection
sql_connection.close()
sqlEngine.dispose()

print ("Connection closed.")

NameError: name 'db_engine' is not defined